In [34]:
import numpy as np
import json
import glob


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# #spacy
import spacy
from nltk.corpus import stopwords

#visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


from gensim.models import TfidfModel


In [35]:
#prepping data
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

    return data


def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [36]:
stopwords = stopwords.words("english") + ["go", "re", "just", "get", "want", "know", "so", "need", "knock", "look", "guy", "work", "say", "let", "come", "here", "make", "see",  "tell", "thing", "talk", "ve", "really"]


In [37]:
# loads descriptions of episodes
episodes = load_data("data/plots.json")

# episode titles in the form of "Season;Episode"
names = [str(episode["Season"]) + ";" + str(episode["No. inseason"])
         for episode in episodes]

# maps episode titles to their index in the names array
names_map = {}
for index in range(len(names)):
    names_map[names[index]] = index

# episode summaries
plots = [episode["plot"] for episode in episodes]

# episode summaries in the form of "Season;Episode\nSummary"
summaries = [str(episode["Season"]) + ";" + str(episode["No. inseason"]
                                                ) + "\n" + str(episode["plot"]) for episode in episodes]

# loads episode scripts
episodes_data = {}
data = load_data("data/scripts.json")

for line in data:
    episode_title = line["episode_name"]
    script_line = line["dialogue"]

    if episode_title not in episodes_data:
        episodes_data[episode_title] = ""

    episodes_data[episode_title] += script_line + " "

episodes_array = []
# Iterate through the episodes_data dictionary and extract the script lines
for lines in episodes_data.values():
    # Combine all script lines into a single string for each episode
    episode_string = "".join(lines)
    episodes_array.append(episode_string)

# clean up the episode scripts and combine them with the episode summaries
for i in range(len(episodes_array)):
    episodes_array[i] = episodes_array[i].replace("\\n", " ")
    episodes_array[i] = episodes_array[i].replace("\\", "")
    episodes_array[i] = episodes_array[i].replace("  ", " ")
    plots[i] = plots[i] + " " + episodes_array[i]
    
    

In [50]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    """https://spacy.io/api/annotation"""
    nlp = spacy.load("/Users/anishpalakurthi/opt/anaconda3/lib/python3.8/site-packages/en_core_web_sm/en_core_web_sm-3.6.0", disable=["parser", "ner"])
    texts_out = []
    for sent in texts:
        #contains metadata about the word
        doc = nlp(sent)
        #lemmatizes each word by appending allowed tokens from the doc object's metadata
        texts_out.append(
            " ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags]))
        
    for word in texts_out:
        if word in stopwords:
            texts_out.remove(word)
    return texts_out

print("path successfully hit")
lemmatized_texts = lemmatization(plots)
print(lemmatized_texts[0])

path successfully hit
unsuccessful visit high iq sperm bank return home find aspire actress penny new neighbor hall apartment think immediately interested chase dream never catch invite apartment indian food ask use shower break wrap towel get meet visit friend wannabe lady man try hit unable speak suffer selective mutism presence woman so infatuate help use shower agree retrieve tv ex - boyfriend however physical superiority overwhelm combined iq return pant tv penny feel bad offer take guy dinner initiate friendship corridor sperm bank so photon direct plane slit slit observe go slit ’ unobserved however ’ observe ’ leave plane hit target go slit agree ’ point ’ point just think ’ good idea tee shirt excuse hang across aegean down down move finger phylum make au see capital idea ’ port au help high sperm bank ask maybe here think place fill thank right back take time just finish crossword puzzle wait think kid ’re semi - pro commit genetic fraud ’ guarantee sperm go generate high iq 

In [51]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
        
    return final

#preprocess our lemmatized keywords
data_words = gen_words(lemmatized_texts)

print(data_words[0][0:10])

['unsuccessful', 'visit', 'high', 'iq', 'sperm', 'bank', 'return', 'home', 'find', 'aspire']


In [52]:
#bigrams and trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)


def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)


print(data_bigrams_trigrams[0])


['unsuccessful', 'visit', 'high', 'iq', 'sperm', 'bank', 'return', 'home', 'find', 'aspire', 'actress', 'penny', 'new', 'neighbor', 'hall', 'apartment', 'think', 'immediately', 'interested', 'chase', 'dream', 'never', 'catch', 'invite', 'apartment', 'indian', 'food', 'ask', 'use', 'shower', 'break', 'wrap', 'towel', 'get', 'meet', 'visit', 'friend', 'wannabe', 'lady', 'man', 'try', 'hit', 'unable', 'speak', 'suffer', 'selective_mutism', 'presence', 'woman', 'so', 'infatuate', 'help', 'use', 'shower', 'agree', 'retrieve', 'tv', 'ex_boyfriend', 'however', 'physical', 'superiority', 'overwhelm', 'combined', 'iq', 'return', 'pant', 'tv', 'penny', 'feel', 'bad', 'offer', 'take', 'guy', 'dinner', 'initiate', 'friendship', 'corridor', 'sperm', 'bank', 'so', 'photon', 'direct', 'plane', 'slit', 'slit', 'observe', 'go', 'slit', 'unobserved', 'however', 'observe', 'leave', 'plane', 'hit', 'target', 'go', 'slit', 'agree', 'point', 'point', 'just', 'think', 'good', 'idea', 'tee_shirt', 'excuse', '

In [53]:
#tfidf removal

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []

words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
        
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    
    corpus[i] = new_bow



In [54]:
id2word = corpora.Dictionary(data_words)

corpus = [id2word.doc2bow(text) for text in data_words]

for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

#[index, frequency]
print(corpus[0][0:10])


[(0, 3), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [56]:
#generate LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word = id2word, num_topics=5, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [57]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = "mmds", R = 30)
vis

/Users/anishpalakurthi/Library/Python/3.10/lib/python/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.140364  0.068083       1        1  48.654738
3     -0.117370  0.024662       2        1  40.742986
2     -0.043460 -0.111701       3        1   5.139178
4      0.075075  0.129637       4        1   3.844721
0      0.226118 -0.110681       5        1   1.618377, topic_info=      Term          Freq         Total Category  logprob  loglift
293  knock   1505.000000   1505.000000  Default  30.0000  30.0000
220     go  10975.000000  10975.000000  Default  29.0000  29.0000
401  penny   1722.000000   1722.000000  Default  28.0000  28.0000
217    get   8561.000000   8561.000000  Default  27.0000  27.0000
364   need   1980.000000   1980.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
387   open     15.812540    544.934463   Topic5  -6.3214   0.5839
470    see     17.020895   3556.523414   Topic5  -6.2478  -1.2184
320   look     16.888619   3123.210155   Topic5  -6.2556  -1.0962
220     go     16.559518  10975.517974   Topic5  -6.2752  -2.3727
515  start     15.303331   1059.963694   Topic5  -6.3541  -0.1142

[454 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
9         1  0.517070   again
9         2  0.385518   again
9         3  0.052986   again
9         4  0.026493   again
9         5  0.018271   again
...     ...       ...     ...
7695      1  0.097790  zombie
7695      2  0.317817  zombie
7695      3  0.586739  zombie
7695      5  0.024447  zombie
6237      5  0.771885    zoom

[896 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 5, 1])